In [1]:
from pyspark.sql import functions as F
from pyspark.sql import types
from pyspark.sql import Row

In [2]:
contest_data_sample = sqlContext.sql("select * from edso_ignite.contest_data").sample(False,0.01).repartition(100).cache()
contest_savm = sqlContext.sql("select * from edso_ignite.contest_savm").repartition(200).cache()
contest_cr = sqlContext.sql("select * from edso_ignite.contest_cr").repartition(200).cache()

In [213]:
from math import *
import string
import re

def jaccard_similarity(x,y):
    intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
    union_cardinality = len(set.union(*[set(x), set(y)]))
    if float(union_cardinality) != 0:
        return intersection_cardinality/float(union_cardinality)
    else:
        return 0
    
stop_words = ['GROUP','LLC','INC','LTD','LIMITED','MEDIA','COUNTY','LLP','&','AND','AGENCY','LP','CORPORATION','COMPANY','DEPT','DEPTT','USA','CORP','COMMUNICATIONS','AMERICA','NORTH AMERICA', 'SYSTEMS', 
'FORUM', 'LOGISTICS', 'DEPT', 'SERVICES', 'OF', 'THE', 'NA', 'BRANDS', 'STATE','CORP','ST', 'SELECT', 'INTERNATIONAL','COMMERCIAL','HOLDINGS','HOLDING','ENTERPRISE']
org_identifying_words = ['ATT','VERIZON','ARMY','NAV']

def treat_names(name):
    name = ''.join(ch for ch in name if ch not in string.punctuation)#.split('OFFICE OF')[0].split('ATT')[0].split('VERIZON')[0].split('PO')[0].split('CO')[0] # fastest way to remove punctutation
    resultwords  = [word for word in name.split() if word not in stop_words]
    return ' '.join(resultwords)

def name_matching(a,b):
    if (("NAV" in a and 'NAV' in b) or ("ARMY" in a and 'ARMY' in b)):
        return 1.0
    else:
        return jaccard_similarity(treat_names(a),treat_names(b))
#################
def treat_namesXXX(name):
    return name in org_identifying_words

    

In [214]:
#name_matching('ATT-TELEPRESENCE','ATT INC')
treat_namesXXX('ATT')
#treat_names('ATT INC')
name_matching('US ARMY AVIATION & MISSILE COMMAND','US ARMY SMDC')

1.0

In [215]:
smpl_join = F.broadcast(contest_data_sample).join(contest_cr, F.col('end_customer_party_ssot_party_id_int_sav_party_id')==F.col('party_id'))#.select('party_id','decision_date_time','party_name','postal_code','address1','street_name','sales_acct_id').cache()

In [216]:
smpl_join_r = smpl_join.select(*(F.col(x).alias(x + '_orig') for x in smpl_join.columns))
join_a = F.broadcast(smpl_join_r).join(contest_savm,F.col('postal_code_orig')==F.col('postal_code')).where(F.col('decision_date_time_orig')<=F.col('start_date'))

In [217]:
join_a = join_a.na.fill('a',["party_name_orig","address1_orig","street_name_orig"])
join_a = join_a.na.fill('z',["party_name","address1","street_name"])

In [218]:
jaccard_udf = F.udf(jaccard_similarity,types.DoubleType())
name_matching_udf = F.udf(name_matching,types.DoubleType())
join_b = join_a.withColumn("name_similairity", name_matching_udf(join_a["party_name_orig"],join_a["party_name"]))
join_b = join_b.withColumn("address_similairity", jaccard_udf(join_a["address1_orig"],join_a["address1"]))
join_b = join_b.withColumn("st_name_similairity", jaccard_udf(join_a["street_name_orig"],join_a["street_name"]))

In [219]:
def find_prediction(savm_id,data_id):
    return 1 if savm_id==data_id else 0

find_prediction_udf = F.udf(find_prediction,types.IntegerType())
join_b = join_b.withColumn("predicted", find_prediction_udf(join_b["sales_acct_id_orig"],join_b["sales_acct_id"]))#.select(['endcustomerlinefixed_data','predicted'])

In [220]:
join_b = join_b.na.fill('a',["name_similairity","address_similairity","st_name_similairity"])

In [221]:
builder = join_b.withColumn("name_similarity", F.when(F.col("name_similairity").isNull(), 0).otherwise(F.col("name_similairity")))
builder = builder.withColumn("address_similarity", F.when(F.col("address_similairity").isNull(), 0).otherwise(F.col("address_similairity")))
builder = builder.withColumn("st_name_similarity", F.when(F.col("st_name_similairity").isNull(), 0).otherwise(F.col("st_name_similairity")))

In [222]:
def aggregated_prediction(name,adr,st):
        #return (4.3*name + 1.2*adr+ 0.8*st)
        return name

aggregated_prediction_udf = F.udf(aggregated_prediction,types.DoubleType())
join_c = builder.withColumn("aggregated_prediction", aggregated_prediction_udf(builder['name_similarity'],builder['address_similarity'],builder['st_name_similarity']))#.select(['endcustomerlinefixed_data','predicted'])

In [223]:
#In join_c table, if find duplicate party_id_orig, select the value where aggregated_prediction value is the highest
join_c.registerTempTable("x")
predictions_table = sqlContext.sql("SELECT party_id_orig,aggregated_prediction,predicted  FROM( SELECT *, ROW_NUMBER()OVER(PARTITION BY party_id_orig ORDER BY aggregated_prediction DESC) rn FROM x) y WHERE rn = 1").cache()

In [224]:
predictions_table.count()

4237

In [225]:
predictions_table.where(F.col('predicted')==1).count()

1366

Out of 569 unique party_ids, 212 were correctly predicted. That's accuracy of 37.2%


Out of 4324 unique party_ids, 1426 were correctly predicted. That's accuracy of 32.9%

Out of 4237 unique party_ids, 1328 were correctly predicted. With only name matching

# Exploratory Analysis

In [226]:
join_c.groupby("predicted").agg(F.min("aggregated_prediction"),F.avg("aggregated_prediction"),F.max("aggregated_prediction")).show()

+---------+--------------------------+--------------------------+--------------------------+
|predicted|min(aggregated_prediction)|avg(aggregated_prediction)|max(aggregated_prediction)|
+---------+--------------------------+--------------------------+--------------------------+
|        0|                       0.0|        0.3969497573801709|                       1.0|
|        1|                       0.0|         0.748574793554742|                       1.0|
+---------+--------------------------+--------------------------+--------------------------+



In [227]:
expl = join_c.where((F.col('predicted')==1) & (F.col('aggregated_prediction')<0.7)).sample(False,0.1)
expl.select('party_name_orig','party_name','name_similarity','aggregated_prediction').show(10,False)#.groupby("predicted").agg(F.avg("aggregated_prediction")).show()

+---------------------------------------------------------+-----------------------------------------------+------------------+---------------------+
|party_name_orig                                          |party_name                                     |name_similarity   |aggregated_prediction|
+---------------------------------------------------------+-----------------------------------------------+------------------+---------------------+
|YOUNG MEN'S CHRISTIAN ASSOCIATION OF HONOLULU            |THE RESORT GROUP LLC                           |0.3125            |0.3125               |
|TELCOBUYCOM LLC                                          |AT&T SERVICES INC                              |0.1               |0.1                  |
|DISNEY WORLDWIDE SERVICES INC                            |WALT DISNEY IMAGINEERING                       |0.6666666666666666|0.6666666666666666   |
|RENTA DE EQUIPO SA DE CV                                 |TELEFONOS DE MEXICO SAB DE CV                  

In [153]:
expl.select('address1_orig','address1','address_similairity','aggregated_prediction','predicted').show(10,False)#.groupby("predicted").agg(F.avg("aggregated_prediction")).show()

+--------------------------+-------------------------------+-------------------+---------------------+---------+
|address1_orig             |address1                       |address_similairity|aggregated_prediction|predicted|
+--------------------------+-------------------------------+-------------------+---------------------+---------+
|HEALTH CARE FACILITY      |JUNCTION STATE RD 371 & ROUTE 9|0.3181818181818182 |3.0373737373737373   |1        |
|13101 COLUMBIA PIKE FDC II|13101 COLUMBIA PIKE            |0.8823529411764706 |3.397216386554622    |1        |
|6820 S HARL AVE           |1900 WEST LOOP S STE 1600      |0.3333333333333333 |3.553333333333333    |1        |
|6820 S HARL AVE           |1900 WEST LOOP S STE 1600      |0.3333333333333333 |3.553333333333333    |1        |
|20 VIADUCT HARBOUR AVENUE |SHORTLAND STREET               |0.5                |3.2                  |1        |
|200 N MILWAUKEE AVE       |1140 T ST                      |0.11764705882352941|3.36617647058823

In [154]:
expl.select('street_name_orig','street_name','st_name_similairity','aggregated_prediction','predicted').show(10,False)#.groupby("predicted").agg(F.avg("aggregated_prediction")).show()

+----------------------+-------------------------------+-------------------+---------------------+---------+
|street_name_orig      |street_name                    |st_name_similairity|aggregated_prediction|predicted|
+----------------------+-------------------------------+-------------------+---------------------+---------+
|HIGHWAY 191 HOSPITAL  |JUNCTION STATE RD 371 & ROUTE 9|0.3333333333333333 |3.0373737373737373   |1        |
|COLUMBIA PIKE FDC II  |COLUMBIA                       |0.5714285714285714 |3.397216386554622    |1        |
|HARL                  |WEST                           |0.0                |3.553333333333333    |1        |
|HARL                  |WEST                           |0.0                |3.553333333333333    |1        |
|VIADUCT HARBOUR AVENUE|SHORTLAND STREET               |0.5625             |3.2                  |1        |
|MILWAUKEE             |T                              |0.0                |3.366176470588235    |1        |
|HARL              

In [238]:
def name_matching(a,b):
    if (("NAV" in a and 'NAV' in b) or ("ARMY" in a and 'ARMY' in b)):
        return 1
    else:
        return jaccard_similarity(treat_names(a),treat_names(b))

In [239]:
stop_words = ['GROUP','LLC','INC','LTD','MEDIA','COUNTY','LLP','&','AND','AGENCY','LP','CORPORATION','COMPANY','DEPT','DEPTT','USA','CORP','COMMUNICATIONS','AMERICA','NORTH AMERICA', 'SYSTEMS', 
'FORUM', 'LOGISTICS', 'DEPT', 'SERVICES', 'OF', 'THE', 'NA', 'BRANDS', 'STATE','CORP','ST', 'SELECT', 'INTERNATIONAL']
import string
def treat_names(name):
    name = ''.join(ch for ch in name if ch not in string.punctuation).split('OFFICE OF')[0].split('PO')[0].split('CO')[0] # fastest way to remove punctutation
    resultwords  = [word for word in name.split() if word not in stop_words]
    return ' '.join(resultwords)
    #return name

In [51]:
name_matching('HONDA OF AMERICA MFG INC','HONDA OF AMERICA')

0.5555555555555556

In [61]:
jaccard_similarity('USDOTMARAD','DOTUS')

0.625

In [52]:
treat_names('HONDA OF AMERICA MFG INC')

'HONDA MFG'

3